In [2]:
pip install CurrencyConverter


In [ ]:
'₤375,000 [ 2 ]'},
'£527,530 [ 1 ]
'£527,530 [ 1 ]'
$8–8.5 million [ 2 ] [ 3 ]
£9.8 million ($15 million)


Any non-USD amount should be converted to USD
ii. Any NaN should be converted to 0
iii. Any budget that is a range (ie: $10,000,000 - $20,000,000), should be
removed or $0


In [1]:
import requests
from pydantic import BaseModel, Field
from typing import Optional
import json
from currency_converter import CurrencyConverter
import pandas as pd

In [292]:
# def pounds_to_dollars(pounds: int):
#     c = CurrencyConverter()
#     conv = c.convert(pounds, 'GBP', 'USD')
    
#     return int(conv)

In [383]:
def filter_oscar_winners(result: dict) -> list:
    winners = []
    for data in result['results']:
        year = data['year']
        for film in data['films']:
            if film['Winner']:
                film['Year'] = year
                winners.append(film)
                break
    
    return winners

In [384]:
def retrieve_data(endpoint: str = 'http://oscars.yipitdata.com/') -> list:
    result = requests.get(f'{endpoint}').json()
    
    filtered_data = filter_oscar_winners(result)
    
    return filtered_data

In [353]:
# r = requests.get('http://oscars.yipitdata.com/')

In [360]:
# result = r.json()

In [385]:
# a = retrieve_data()

In [179]:
# for data in result['results']:
#     #print(data)
#     year = data['year']
#     for film in data['films']:
#         if film['Winner']:
#             film['Year'] = year
#             winners.append(film)
#             break

In [181]:
def get_budget_from_api(detail_url: str):
    try:
        response = requests.get(f'{detail_url}')
        if 'Budget' in response.json(): 
            budget = response.json()['Budget']
        else: 
            budget = '0'
    except: 
        budget = '0'
    
    return budget

In [320]:
def to_million(cleaned: str):
    return int(float(cleaned) * 1000000)

In [390]:
def process_budget_data(film: Film):
    
    if '–' in film.Budget or '-' in film.Budget:
        film.OriginalBudget, film.USDBudget = 0, 0
        
    elif film.Budget.startswith("US$ "):
        cleaned = film.Budget.split(' ')[1]
        if 'million' in film.Budget:
            film.OriginalBudget, film.USDBudget = to_million(cleaned), to_million(cleaned)
        else: film.OriginalBudget, film.USDBudget = int(cleaned), int(cleaned)
        
    elif film.Budget.startswith("US$"):
        cleaned = film.Budget.split(' ')[0].replace("US$", "")
        if 'million' in film.Budget:
            film.OriginalBudget, film.USDBudget = to_million(cleaned), to_million(cleaned)
        else: film.OriginalBudget, film.USDBudget = int(cleaned), int(cleaned)
        
    elif film.Budget.startswith("$"):
        cleaned = film.Budget.split(' ')[0].replace("$", "")
        if 'million' in film.Budget:
            film.OriginalBudget, film.USDBudget = to_million(cleaned), to_million(cleaned)
        else: 
            film.OriginalBudget, film.USDBudget = int(cleaned), int(cleaned)
            
    elif film.Budget.startswith("£"):
        cleaned = film.Budget.split(' ')[0].replace("£", "")
        if 'million' in film.Budget:
            film.OriginalBudget = to_million(cleaned)
            film.USDBudget = pounds_to_dollars(to_million(cleaned))
        else: 
            film.OriginalBudget = int(cleaned)
            film.USDBudget = pounds_to_dollars(int(cleaned))
            
    else: film.OriginalBudget, film.USDBudget = 0, 0

In [391]:
class Film(BaseModel):
    DetailUrl: str = Field(alias='Detail URL')
    Film: str
    Producer: str = Field(alias='Producer(s)')
    ProductionCompany: str = Field(alias='Production Company(s)')
    WikiURL: str = Field(alias='Wiki URL')
    Winner: bool
    Year: str
    Budget: Optional[str] = ''
    OriginalBudget: Optional[int] = 0
    USDBudget: Optional[int] = 0
    
    def set_budget(self):
        self.Budget = get_budget_from_api(self.DetailUrl)
        if '\xa0' in self.Budget:
            self.Budget = self.Budget.replace('\xa0', ' ')
        elif ',' in self.Budget:
            self.Budget = self.Budget.replace(',', '')
        
    def clean_budget(self):
        process_budget_data(self)
            
    def clean_year(self):
        year = self.Year.strip().split(" ")[0]
        self.Year = year
        
    def __init__(self, **data):
        super().__init__(**data)
        self.set_budget()
        self.clean_budget()
        self.clean_year()

In [ ]:
def to_csv(winners: list):
    out = []
    for winner in winners:
        obj = Film(**winner)
        obj = obj.model_dump_json()
        input_dict = json.loads(obj)
        out.append(input_dict)
    
    dfItem = pd.DataFrame.from_records(out)
    df = dfItem[['Film', 'Year', 'Winner', 'OriginalBudget', 'USDBudget']]
    df.to_csv('test.csv', index=False, encoding='utf-8')

In [ ]:
#film, year, wikipedia_url, Oscar winner, original budget, and budget converted to USD

In [398]:
df

,Film,Year,Winner,OriginalBudget,USDBudget
0,Wings,1927,True,2000000,2000000
1,The Broadway Melody,1928,True,0,0
2,All Quiet on the Western Front,1929,True,1200000,1200000
3,Cimarron,1930,True,1433000,1433000
4,Grand Hotel,1931,True,750000,750000
...,...,...,...,...,...
82,The King's Speech,2010,True,15000000,15000000
83,The Artist,2011,True,15000000,15000000
84,Argo,2012,True,44500000,44500000
85,12 Years a Slave,2013,True,22000000,22000000


In [ ]:
# if '–' in self.Budget or '-' in self.Budget:
#             self.OriginalBudget, self.USDBudget = 0, 0
#         elif self.Budget.startswith("US$ "):
#             cleaned = self.Budget.split(' ')[1]
#             if 'million' in self.Budget:
#                 self.OriginalBudget, self.USDBudget = to_million(cleaned), to_million(cleaned)
#             else: self.OriginalBudget, self.USDBudget = int(cleaned), int(cleaned)
#         elif self.Budget.startswith("US$"):
#             cleaned = self.Budget.split(' ')[0].replace("US$", "")
#             if 'million' in self.Budget:
#                 self.OriginalBudget, self.USDBudget = to_million(cleaned), to_million(cleaned)
#             else: self.OriginalBudget, self.USDBudget = int(cleaned), int(cleaned)
#         elif self.Budget.startswith("$"):
#             cleaned = self.Budget.split(' ')[0].replace("$", "")
#             if 'million' in self.Budget:
#                 self.OriginalBudget, self.USDBudget = to_million(cleaned), to_million(cleaned)
#             else: 
#                 self.OriginalBudget, self.USDBudget = int(cleaned), int(cleaned)
#         elif self.Budget.startswith("£"):
#             cleaned = self.Budget.split(' ')[0].replace("£", "")
#             if 'million' in self.Budget:
#                 self.OriginalBudget = to_million(cleaned)
#                 self.USDBudget = pounds_to_dollars(to_million(cleaned))
#             else: 
#                 self.OriginalBudget = int(cleaned)
#                 self.USDBudget = pounds_to_dollars(int(cleaned))